In [1]:
import scripts.exist2024
import pandas as pd

#datasets = scripts.exist2024.load_dataset('EXIST 2024 Memes Dataset')
#test_dataset = datasets['f_test_dataset.json'].encode_test("EXIST 2024 Memes Dataset", "test", "memes")
#test_dataset_with_embeddings = test_dataset.find_embeddings()
#test_dataset_with_embeddings
#test_dataset_with_embeddings.to_parquet('test_vectorspace.parquet')
test_dataset = pd.read_parquet('test_vectorspace.parquet')
test_dataset

In [2]:
vectorstore = scripts.exist2024.load_vectorstore()
vectorstore

In [4]:
task4_model = vectorstore.fit("score_task4")
task5_model = vectorstore.fit("score_task5")
task6_model = vectorstore.fit("score_task6")

In [6]:
X_test = test_dataset.predictor_variable()

In [7]:
test_dataset_with_predictions = test_dataset.assign(
    score_task4=task4_model.predict(X_test),
    score_task5=task5_model.predict(X_test),
    score_task6=task6_model.predict(X_test)
)
test_dataset_with_predictions

In [12]:
test_dataset_with_predictions.to_parquet("test_dataset_with_annotator_item_preferences")

# Find labels

In [2]:
import scripts.exist2024
import pandas as pd

test_dataset_with_predictions = pd.read_parquet("test_dataset_with_annotator_item_preferences")
n = 1

In [3]:
def ut4(df):
    matrix = df.pivot(index='annotator', columns='item_id', values="score_task4")
    matrix.loc['Voting'] = matrix.mean()
    matrix.loc['YES'] = matrix.loc['Voting'].clip(0, 1)
    matrix.loc['NO'] =  1-matrix.loc['Voting']
    matrix.loc['Hard'] =  matrix.loc[['YES', 'NO']].idxmax()
    return matrix

import json

def write_json(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f)

n = 100
dt4 = ut4(test_dataset_with_predictions)
result = [{"test_case": "EXIST2024", "id": col, "value": dt4.at['Hard', col]} for col in dt4.columns]
task4_hard_MMICI_3 = [{"test_case": "EXIST2024", "id": col, "value": dt4.at['Hard', col]} for col in dt4.columns]
task4_soft_MMICI_3 = [{"test_case": "EXIST2024", "id": col, "value": {
    'YES': dt4.at['YES', col],
    'NO': dt4.at['NO', col],
}} for col in dt4.columns]

write_json('task4_hard_MMICI_3.json', task4_hard_MMICI_3)
write_json('task4_soft_MMICI_3.json', task4_soft_MMICI_3)

In [5]:
dt4['310001']

In [128]:

def g(column):
    t = pd.cut(x=column, bins=[0,0.75,1.5,3], labels=["NO", "DIRECT", "JUDGEMENTAL"]).value_counts(sort=False) 
    exp_df = np.exp(t - np.max(t))
    sum_of_exp = exp_df.sum()
    softmax_df = exp_df / sum_of_exp
    return pd.concat([column, softmax_df])
    
def ut5(df):
    matrix = df.pivot(index='annotator', columns='item_id', values="score_task5")
    matrix_with_probs = matrix.apply(g)
    matrix_with_probs.loc['Hard'] =  matrix_with_probs.loc[['NO', 'DIRECT', "JUDGEMENTAL"]].idxmax()
    return matrix_with_probs

n = 100
dt5 = ut5(test_dataset_with_predictions)

task5_hard_MMICI_3 = [{"test_case": "EXIST2024", "id": col, "value": dt5.at['Hard', col]} for col in dt5.columns]
task5_soft_MMICI_3 = [{"test_case": "EXIST2024", "id": col, "value": {
    'NO': dt5.at['NO', col],
    'DIRECT': dt5.at['DIRECT', col],
    'JUDGEMENTAL': dt5.at['JUDGEMENTAL', col],
}} for col in dt5.columns]

write_json('task5_hard_MMICI_3.json', task5_hard_MMICI_3)
write_json('task5_soft_MMICI_3.json', task5_soft_MMICI_3)

In [15]:
import scripts.exist2024 as exist2024

exist2024.categories[-2] = "NO"
bits = {2**index:category  for index, category in enumerate(exist2024.categories)}

def decode(bitset):
    return [value for key, value in bits.items() if key & bitset]

assert decode(2**1) == ['STEREOTYPING-DOMINANCE']
assert decode(2**0) == ['IDEOLOGICAL-INEQUALITY']
assert decode(3) == ['IDEOLOGICAL-INEQUALITY', 'STEREOTYPING-DOMINANCE']
assert decode(0b000101) == ['IDEOLOGICAL-INEQUALITY', 'OBJECTIFICATION']

In [16]:
categories = exist2024.categories[:-1]
categories

In [8]:
def g(column):
    column_rounded = column.dropna().round().astype(int)
    counter = [0]*len(categories)
    for key, value in enumerate(column_rounded):
     i = 0
     while column_rounded.iloc[key] > 0:
       counter[i] += column_rounded.iloc[key] & 1
       column_rounded.iloc[key] = column_rounded.iloc[key] >> 1
       i = i+1
    for index, category in enumerate(categories):
        column[category] = counter[index]/len(categories)
    column['Hard'] = column[categories][column[categories] >= 0.5].index.values
    return column
    


In [92]:
def g(column):
    print(column)
    column_rounded = column.round().astype(int)
    return column
    


In [17]:
def ut6(df):
    matrix = df.pivot(index='annotator', columns='item_id', values="score_task6")
    matrix = matrix.apply(g)
    return matrix

n = 100
dt6 = ut6(
    test_dataset_with_predictions.assign(
      score_task6=test_dataset_with_predictions['score_task6'].clip(0,32)
    )
)

dt6

In [110]:
dt6.loc['SEXUAL-VIOLENCE'].value_counts().plot(kind='bar')

In [135]:
task6_hard_MMICI_3 = [{"test_case": "EXIST2024", "id": col, "value": list( dt6.at['Hard', col])} for col in dt6.columns]

task6_soft_MMICI_3 = [{"test_case": "EXIST2024", "id": col, "value": {
    category: dt6.at[category, col] for category in categories
}} for col in dt6.columns]

write_json('task6_hard_MMICI_3.json', task6_hard_MMICI_3)
write_json('task6_soft_MMICI_3.json', task6_soft_MMICI_3)

In [123]:
dt5.loc['Hard'].value_counts().plot(kind='bar')

In [18]:
test_dataset_with_predictions.plot.hist(column=["score_task4"], figsize=(10, 8))

In [9]:
test_dataset_with_predictions.plot.hist(column=["score_task5"], figsize=(10, 8))

In [184]:
test_dataset_with_predictions[['score_task6']].clip(0,32).max()

In [8]:
test_dataset_with_predictions.plot.hist(column=["score_task6"], figsize=(10, 8))